# Iterated conditional probability kernel computation 
### Create_expanding_rings makes the kernel. 
Claude wrote the template code from which this was "evolved".  

Create a square arena, and iterate cell formation according to an expanding-rings conditional probability, modeled on the IR satellite result over South America. 

A main question is: Do large-scale and low frequency patterns emerge from a conditional p' pattern that is bounded in space and time? Call such a bounded pattern a "KERNEL". 

One important parameter for larger scales might seem to be the NET probability enhancement over the kernel. If this is positive, clumping can run away at the kernel's bounding-box scales as in the Claude_draft notebook example with its positive-only kernel. If negative, we might get oscillating exclusions, either blue noise speckle or perhaps clumped shaped by that kernel outer scale boundary and/or the domain periodic boundaries. Neither of these degenerate cases is of interest here, so let's go for a neutral kernel, with the sum of log-probability enhancements equal to zero over the whole kernel. 

A probelm with an additive probability formulation is that p must remain nonnegative. But if log-kernel is our fit to the expanding rings seen in the data, then 2**kernel is a nonnegative factor we can multiply by a background probability. Whatever the pattern, the number of cells in the domain (set by the spatial mean probability) should be kept constant, a stable climatology. The clusters and patterns should not be allowed to run away or collapse. This does require an adjustment after all the kernels are multiplied, which can result in an instantaneous suppression of all the areas beyond an organized structure that lines up all its kernel factors to multiply bigly. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2048  # set to 2 GB (2,097,152 bytes)

In [ ]:
# DEFINE SQUARE ARENA IN WHICH THEY INTERACT

# Constants
SIZE = 300
PMEAN = 0.01         # 0.001 means ~250 new cells each step in 500x500 array 

# Initialize probability map
pbar = np.full((SIZE, SIZE), PMEAN)
pcondfactor = np.zeros_like(pbar) + 1 
prob = pbar*pcondfactor

# Initialize grid of cell age at zero
# They will age until overwritten with 1 at birth. 
cell_age = np.zeros((SIZE, SIZE), dtype=int)

In [ ]:
# DEFINE KERNEL 
# In the paper, values in [-0.3,0.3] are log2 of a probability enhancement. 
# Multiply kernel = 2**kernel by pmean to get a total probability field. 
# Multiplying rather than adding/subtracting will keep prob positive 

logkernel = np.load('/Users/bmapes/Github/EvolutionaryConvection/Probability_kernel_iteration/logpkernel_21x21x5.numpy.npy')
kernel = 2**logkernel

KERNEL_SIZE = kernel.shape[2]      # 21x21 gridpoints, symmetric on a central point
KERNEL_TIME = kernel.shape[0]      # number of cell lifetimes that pcondfactor is affected

# ASYMMETRY: leftward propagation, should probably do cos(angle) not just left:right half/half. 
kernel[:,:,:11] *= 1.05
kernel[:,:,11:] *= 0.95

plt.figure(figsize=(6,2.5))
plt.pcolormesh(np.linspace(-10,10,21), np.linspace(1,5,5), kernel[:,10,:], \
               cmap='RdBu_r',vmin=0.3, vmax=1.7); plt.colorbar();
# plt.contour(np.linspace(-10,10,21), np.linspace(1,5,5), kernel[:,10,:], levels=10)
plt.ylabel('hours'), plt.xlabel('distance from cell at time 0');
plt.title('Conditional probability factor after t=0 cell');

In [ ]:
plt.pcolormesh(kernel[3,:,:], cmap='RdBu_r',vmin=2**(-0.3), vmax=2**0.3); 
plt.colorbar();

In [ ]:
print( np.mean(logkernel), ' mean logkernel value being subtracted' )
logkernel -= np.mean(logkernel)
print( np.mean(logkernel), ' mean logkernel value was subtracted' )

In [ ]:
# KERNEL EXPERIMENTS 

# One time duration: Markovian in time 
#KERNEL_TIME = 1 

#small kernel, net positive: makes "MCS" about 50 pixels 
#kernel = 2**kernel[4,:,:]

# Bigger pattern, strong hole in the middle. Hole makes blobs w cavities @ CLIPFACTOR=20
# kernel = 2**kernel[3,:,:]

# DIVIDE BY the area-mean of the FACTOR - so area average is muted: no org just ripples
#kernel /= (np.mean(kernel))

# ASYMMETRY: yes, leftward propagation
#kernel[:,:10] *= 1.05
#kernel[:,10:] *= 0.95

#plt.pcolormesh( kernel ); plt.colorbar();

In [ ]:
# Update(frame) function does one timestep evolution, np.random.random cells

def update(frame):
    # global variables: not best practice, but lucid for debugging and playing  
    global new_cells, cell_age, pbar, pcondfactor, prob, CLIPFACTOR, DIURNAL_AMP   
    
    # Age all existing cells by 1, zeros remain unchanged.
    cell_age = np.where(cell_age>0.99, cell_age+1, cell_age)

    # Create new cells based on probability map (Boolean)
    new_cells = np.random.random((SIZE, SIZE)) < (prob)
    
    # Overwrite cell_age with 1 where a new cell is located 
    cell_age = np.where(new_cells, 1, cell_age) 

    
    # Update total probability map, prob = pbar * pcondfactor    
    
    # pcondfactor: Loop over KERNEL_TIME, multiplying kernel from cells of that age 
    pcondfactor = np.zeros_like(pbar) + 1.0
    
    for iage in range(KERNEL_TIME): 
        age = iage+1 # 
        active_cells = np.where( (cell_age==age) )
        for i, j in zip(*active_cells):
            i_start = (i - KERNEL_SIZE // 2) % SIZE
            j_start = (j - KERNEL_SIZE // 2) % SIZE
            i_indices = np.arange(i_start, i_start + KERNEL_SIZE) % SIZE
            j_indices = np.arange(j_start, j_start + KERNEL_SIZE) % SIZE
            # A patch of kernel application
            pcondfactor[np.ix_(i_indices, j_indices)] *= kernel[iage,:,:]

    prob = pbar*pcondfactor 
    
    # Clip values, it does have a runaway problem with positivity with KERNEL_TIME=1
    prob = np.clip(prob, 0, CLIPFACTOR*PMEAN)

    # Rescale total prob to PMEAN, prevent drift of total amount. WTG-like instant subsidence 
    prob *= PMEAN/np.mean(prob)
    
    # prob has a diurnal cycle, must not exceed 1 to keep probability nonnegative 
    if (DIURNAL_AMP>=1):
        DIURNAL_AMP=0.99
    prob *= ( 1 + DIURNAL_AMP * np.sin(2*np.pi *frame/24) ) 

    # Update image : mean age version
    # im.set_array(cell_age)
    # plt.title(f"Step: {frame}, Mean age of cells: {np.mean(cell_age)}")

    # Update image: prob version 
    im.set_array(prob)
    plt.title(f"Step: {frame}")

    # Print debugging information
    #print(f"Step {frame}: Min prob = {prob.min():.6f}, Max prob = {prob.max():.6f}, Mean prob = {np.mean(prob)}")
    
    return [im]

In [ ]:
# RUN AND ANIMATE RESULTS 
TIMESTEPS = 24*2 # 5.5 days, ending during daytime if diurnal amplitude is nonzero
CLIPFACTOR = 100  # how high above PMEAN can the local p be allowed to get? 
                  # Prevents runaway storms, shouldn't be needed, will see it if it is 
DIURNAL_AMP = 0   # background probability constant in time

# Initialize probability map
pbar = np.full((SIZE, SIZE), PMEAN)
pcondfactor = np.zeros_like(pbar) + 1. 
prob = pbar * pcondfactor

# Initialize grid of cell age
cell_age = np.zeros((SIZE, SIZE), dtype=int)

fig, ax = plt.subplots(figsize=(6,4))
im = ax.pcolormesh(prob, vmin=0, vmax=PMEAN*3)
plt.colorbar(im)
plt.title("Cellular Automaton")

# Create the animation
anim = FuncAnimation(fig, update, frames=TIMESTEPS, interval=50, repeat=False)

# Display the animation
HTML(anim.to_jshtml())

In [ ]:
# Save the animation as an MP4 file: it sucks 
anim.save('EvoCA_animation.mp4', writer='ffmpeg',fps=60, extra_args=['-vf', 'scale=-1:-1'], codec='h264', bitrate=1000000) 

In [ ]:
prob.max()/PMEAN  # Is it hitting CLIPFACTOR? Hopefully not 

In [ ]:
# whole past history is stored here

from scipy.ndimage import gaussian_filter as smoo 

plt.pcolormesh(smoo(cell_age,0), cmap='jet'); plt.colorbar(); 
plt.title('Time since last cell init, 84h no diurnal cycle');

# From a map of cell age, convective+stratiform depiction

Old cells create expanding but decayihg stratiform anvils. Mimic this graphically 

In [ ]:
def ageplot(age, MINAGE=1, MAXAGE=8, NAGES=5): 
    
    colorz= ['cyan','blue','green','orange','red','magenta'] 

# Few hour age range, 8-6-4-1. For each age, smooth (cells within age/2) at scale (age). 
    
    for iage,age in enumerate( np.flip(np.linspace(MINAGE,MAXAGE,NAGES)) ): 
        print(iage, age-1)

        # make a Boolean True*1.0 of all the cells within age/2 of age
        # and smooth that array by age pixels, like clouds diffusing
        plt.contour( smoo( (np.abs(cell_age-age)<age/2.)*1.0 , age-2), colors=colorz[iage], levels=20, alpha=0.5 )
                     

In [ ]:
# stratiform + convective type of graphic 
ageplot(cell_age, 2,10,5)

# Multiply pmean by a diurnal cycle: treat timestep as 1 hour
### Let's give PMEAN a 24 timestep sine wave

In [ ]:
# RUN AND ANIMATE RESULTS, END DURING night or add 12 to end during day 
TIMESTEPS = 3*24 + 12 +6 
# CLIPFACTOR = 50  # how high above PMEAN can the local p get? Prevents runaway storms 
DIURNAL_AMP = 1  # >1 => background probability goes to 0 at night  # PAPER FIGS USED 1 HERE
                    # If night is longer than KERNEL_TIME --> fresh start every day

# Initialize probability map
pbar = np.full((SIZE, SIZE), PMEAN)
pcondfactor = np.zeros_like(pbar) + 1. 
prob = pbar * pcondfactor

# Initialize grid of cell age
cell_age = np.zeros((SIZE, SIZE), dtype=int)

fig, ax = plt.subplots(figsize=(6,4))
im = ax.pcolormesh(prob, vmin=0, vmax=PMEAN*3)
plt.colorbar(im)
#im = ageplot(cell_age)
# plt.title("Cellular Automaton")

# Create the animation
anim = FuncAnimation(fig, update, frames=TIMESTEPS, interval=50, repeat=False)

# Display the animation
HTML(anim.to_jshtml())

In [ ]:
plt.pcolormesh(cell_age, cmap='jet'); plt.colorbar(); 
plt.title(str(TIMESTEPS)+' h with diurnal amplitude 100%');

In [ ]:
ageplot(cell_age, 2,10,5)